In [45]:
import pandas as pd
import numpy as np

Tabella dei fondi e dei tagli che devi applicare per ottenere i numeri finali.

Dataset         Events/day      Selection_eff   Muon_cut    Fiducial_volume     Combined_efficiency\
Reactor         57.4            98.4%           91.6%       91.2%               82%\
Geo-U           1.155           98.4%           91.6%       91.2%               82%\
Geo-Th          0.345           98.4%           91.6%       91.2%               82%\
Li9             0.81            98.4%           -           91.2%               90%\
He8             0.09            98.4%           -           91.2%               90%\
World_reactors  1.22            98.4%           91.6%       91.2%               82%\
Atmospheric_nu  0.2             98.4%           91.6%       91.2%               82% (?)\
Fast_neutron    0.12            98.4%           91.6%       91.2%               82%\
CO_alpha_n      0.06            98.4%           91.6%       91.2%               82% (?)\
Accidentals     8.8e6           1.41e-6         91.6%       -                   1.29e-6\

Puoi farlo per ogni categoria, poi sommi tutti gli eventi finali e fai:\
efficiency = selected_IBDs/overall_IBDs\
purity = selected_IBDs/selected_events\

In [31]:
import pandas as pd

data = {
    'Reactor': {
        'evday': 57.4,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Geo-U': {
        'evday': 1.155,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Geo-Th': {
        'evday': 0.345,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Li9': {
        'evday': 0.81,
        'selection_eff': 0.984,
        'muon_cut': 1,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.9
    },
    'He8': {
        'evday': 0.09,
        'selection_eff': 0.984,
        'muon_cut': 1,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.9
    },
    'World_reactors': {
        'evday': 1.22,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Atmospheric_nu': {
        'evday': 0.2,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Fast_neutron': {
        'evday': 0.12,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'CO_alpha_n': {
        'evday': 0.06,
        'selection_eff': 0.984,
        'muon_cut': 0.916,
        'fiducial_volume': 0.912,
        'combined_efficiency': 0.82
    },
    'Accidentals': {
        'evday': 8.8e6,
        'selection_eff': 1.41e-6,
        'muon_cut': 0.916,
        'fiducial_volume': 1,
        'combined_efficiency': 1.29e-6
    }
}

df = pd.DataFrame.from_dict(data, orient='index')

#Ricorda, hai messo 1 ai fiducial volume che mancano


In [63]:
eff_manual_cut_IBD = 0.9773
eff_xgboost_IBD = 0.999972
eff_pytorch_IBD =  0.999827


eff_manual_cut_BKG = 0.99997
eff_xgboost_BKG = 0.999985
eff_pytorch_BKG =  0.999819

In [55]:
# eff_manual_cut_IBD = 0.9773
acc_xgboost = 0.9999775018560969 #è l'accuracy del modello
acc_pytorch =  0.999812515467474 #è l'accuracy del modello

In [32]:
df

,evday,selection_eff,muon_cut,fiducial_volume,combined_efficiency
Reactor,57.400,0.984000,0.916,0.912,0.820000
Geo-U,1.155,0.984000,0.916,0.912,0.820000
Geo-Th,0.345,0.984000,0.916,0.912,0.820000
Li9,0.810,0.984000,1.000,0.912,0.900000
He8,0.090,0.984000,1.000,0.912,0.900000
World_reactors,1.220,0.984000,0.916,0.912,0.820000
Atmospheric_nu,0.200,0.984000,0.916,0.912,0.820000
Fast_neutron,0.120,0.984000,0.916,0.912,0.820000
CO_alpha_n,0.060,0.984000,0.916,0.912,0.820000
Accidentals,8800000.000,0.000001,0.916,1.000,0.000001


In [59]:
n_IBD_xgboost = np.array([])
n_IBD_pytorch = np.array([])
all_IBD = np.array([])

for name in df.index.tolist():
    n_IBD_pytorch = np.append(n_IBD_pytorch,df["evday"][name]*acc_pytorch*df["muon_cut"][name])
    n_IBD_xgboost = np.append(n_IBD_xgboost, df["evday"][name]*acc_xgboost*df["muon_cut"][name])

    all_IBD = np.append(all_IBD, df["evday"][name])
    print(df["evday"][name]*acc_pytorch*df["muon_cut"][name],"\t", df["evday"][name]*acc_xgboost*df["muon_cut"][name])
    

sel_IBD_pytorch = n_IBD_pytorch.sum()
sel_IBD_xgboost = n_IBD_xgboost.sum()

all_IBD_n = all_IBD.sum()

print("---------------", '\t', "---------------")
print("Eventi che mi aspetto di selezizonare: ", sel_IBD_pytorch, '\t', sel_IBD_xgboost)

52.568542363255034 	 52.57721708359061
1.0577816451142783 	 1.0579561974137135
0.3159607511380311 	 0.31601289013656375
0.8098481375286539 	 0.8099817765034385
0.08998312639207265 	 0.08999797516704872
1.1173104822852116 	 1.1174948578742254
0.18316565283364125 	 0.18319587834003695
0.10989939170018474 	 0.10991752700402217
0.05494969585009237 	 0.054958763502011086
8059288.724680214 	 8060618.646961627
--------------- 	 ---------------
Eventi che mi aspetto di selezizonare:  8059345.03212146 	 8060674.963694576


Questi appena calcolati dovrebbero essere gli eventi che mi ASPETTO DI SELEZIONARE per un giorno di presa dati.\
Gli algoritmi sviluppati restituiscono una accuracy (Che dovrebbe essere l'efficency come la intendiamo noi, ovvero TP/(TP+FN)). Questa accuracy dovrebbe tener conto dei fattori di fiducial volume e anche dei criteri di selection per come è stato valutato l'apprendimento. Dunque infine quello che abbiamo fatto è ottenere un valore migliorato di 'fiducial_volume' * 'selection_effects'. \
Il muon cut non l'abbiamo proprio considerato, dunque lo teniamo per calcolare gli IBD attesi.

Mi mancano i selected events. Ho una efficency per gli eventi di BKG, devo usare quella?

In [ ]:
# sel_ev = ??

In [62]:
final_efficency_pytorch = sel_IBD_pytorch/all_IBD_n
final_efficency_xgboost = sel_IBD_xgboost/all_IBD_n

# final_purity_pytorch = sel_IBD_pytorch/sel_events

print(final_efficency_pytorch)
print(final_efficency_xgboost)

0.9158282727574446
0.9159794002908407
